<a href="https://colab.research.google.com/github/Ch-rode/CoNLL/blob/main/CoNLL2003_SPACY2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#import io

In [ ]:
#pip install spacy


In [ ]:
#doc = pd.read_csv(io.BytesIO(uploaded['train_notags.csv']),error_bad_lines=False,lineterminator=('\r'),engine="c",sep="\n")
#doc='gdrive/My Drive/TESI/SPACY/train_notags.csv'
#doc

In [ ]:
#from nltk.corpus.reader import ConllCorpusReader
#doc = ConllCorpusReader('gdrive/My Drive/TESI/SPACY/train_notags.txt', 'eng.train', ['words', 'pos', 'ignore', 'chunk'])


In [6]:
import pandas as pd 
doc=pd.read_csv('gdrive/My Drive/LICEO-UNIVERSITA/TESI MAGISTRALE/SPACY/train_notags.csv',error_bad_lines=False,lineterminator=('\r'),engine="c",sep="\n")

FileNotFoundError: ignored

# Named Entity Recognition (NER)
spaCy has an **'ner'** pipeline component that identifies token spans fitting a predetermined set of named entities. These are available as the `ents` property of a `Doc` object.

In [ ]:
import spacy
spacy.__version__

'2.2.4'

In [ ]:
# Perform standard imports
nlp = spacy.load('en_core_web_sm')

In [ ]:
tokens= []
lemma = []
pos = []
tag = []
dep = []
ent= []
label = []
ent_exp= []


for item in nlp.pipe(doc['-DOCSTART-'].astype('str').values, batch_size=50,
                        n_threads=3):
    if item.is_parsed:
        tokens.append([n.text for n in item])
        lemma.append([n.lemma_ for n in item])
        pos.append([n.pos_ for n in item])
        tag.append([n.tag_ for n in item])
        dep.append([n.dep_ for n in item])
        ent.append([item.ents for ent in item.ents])
        label.append([ent.label_ for ent in item.ents])
        ent_exp.append([str(spacy.explain(ent.label_))  for ent in item.ents])

        

    else:
        # We want to make sure that the lists of parsed results have the
        # same number of entries of the original Dataframe, so add some blanks in case the parse fails
        tokens.append(None)
        lemma.append(None)
        pos.append(None)
        tag.append(None)
        dep.append(None)
        ent.append(None)
        label.append(None)
        ent_exp.append(None)
        

doc['doc_token'] = tokens
doc['doc_lemma'] = lemma
doc['doc_pos'] = pos
doc['doc_tag'] = tag
doc["doc_dep"] = dep
doc['doc_ent']= ent
doc['doc_label'] = label
doc['ent_exp'] = ent_exp



In [ ]:
doc.head()

,-DOCSTART-,doc_token,doc_lemma,doc_pos,doc_tag,doc_dep,doc_ent,doc_label,ent_exp
NaN,EU,[EU],[EU],[PROPN],[NNP],[ROOT],"[((EU),)]",[ORG],"[Companies, agencies, institutions, etc.]"
NaN,rejects,[rejects],[reject],[NOUN],[NNS],[ROOT],[],[],[]
NaN,German,[German],[German],[PROPN],[NNP],[ROOT],"[((German),)]",[NORP],[Nationalities or religious or political groups]
NaN,call,[call],[call],[NOUN],[NN],[ROOT],[],[],[]
NaN,to,[to],[to],[ADP],[IN],[ROOT],[],[],[]


In [ ]:
doc[["-DOCSTART-","doc_ent","doc_label","ent_exp"]].head(25)

,-DOCSTART-,doc_ent,doc_label,ent_exp
NaN,EU,"[((EU),)]",[ORG],"[Companies, agencies, institutions, etc.]"
NaN,rejects,[],[],[]
NaN,German,"[((German),)]",[NORP],[Nationalities or religious or political groups]
NaN,call,[],[],[]
NaN,to,[],[],[]
NaN,boycott,[],[],[]
NaN,British,"[((British),)]",[NORP],[Nationalities or religious or political groups]
NaN,lamb,[],[],[]
NaN,.,[],[],[]
NaN,Peter,"[((Peter),)]",[PERSON],"[People, including fictional]"


In [ ]:
doc.nlp=nlp.pipe(doc['-DOCSTART-'].astype('unicode').values, batch_size=50,n_threads=3)
doc.nlp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


<generator object Language.pipe at 0x7f3567fa6af0>

## Entity annotations
`Doc.ents` are token spans with their own set of annotations.
<table>
<tr><td>`ent.text`</td><td>The original entity text</td></tr>
<tr><td>`ent.label`</td><td>The entity type's hash value</td></tr>
<tr><td>`ent.label_`</td><td>The entity type's string description</td></tr>
<tr><td>`ent.start`</td><td>The token span's *start* index position in the Doc</td></tr>
<tr><td>`ent.end`</td><td>The token span's *stop* index position in the Doc</td></tr>
<tr><td>`ent.start_char`</td><td>The entity text's *start* index position in the Doc</td></tr>
<tr><td>`ent.end_char`</td><td>The entity text's *stop* index position in the Doc</td></tr>
</table>



## NER Tags
Tags are accessible through the `.label_` property of an entity.
<table>
<tr><th>TYPE</th><th>DESCRIPTION</th><th>EXAMPLE</th></tr>
<tr><td>`PERSON`</td><td>People, including fictional.</td><td>*Fred Flintstone*</td></tr>
<tr><td>`NORP`</td><td>Nationalities or religious or political groups.</td><td>*The Republican Party*</td></tr>
<tr><td>`FAC`</td><td>Buildings, airports, highways, bridges, etc.</td><td>*Logan International Airport, The Golden Gate*</td></tr>
<tr><td>`ORG`</td><td>Companies, agencies, institutions, etc.</td><td>*Microsoft, FBI, MIT*</td></tr>
<tr><td>`GPE`</td><td>Countries, cities, states.</td><td>*France, UAR, Chicago, Idaho*</td></tr>
<tr><td>`LOC`</td><td>Non-GPE locations, mountain ranges, bodies of water.</td><td>*Europe, Nile River, Midwest*</td></tr>
<tr><td>`PRODUCT`</td><td>Objects, vehicles, foods, etc. (Not services.)</td><td>*Formula 1*</td></tr>
<tr><td>`EVENT`</td><td>Named hurricanes, battles, wars, sports events, etc.</td><td>*Olympic Games*</td></tr>
<tr><td>`WORK_OF_ART`</td><td>Titles of books, songs, etc.</td><td>*The Mona Lisa*</td></tr>
<tr><td>`LAW`</td><td>Named documents made into laws.</td><td>*Roe v. Wade*</td></tr>
<tr><td>`LANGUAGE`</td><td>Any named language.</td><td>*English*</td></tr>
<tr><td>`DATE`</td><td>Absolute or relative dates or periods.</td><td>*20 July 1969*</td></tr>
<tr><td>`TIME`</td><td>Times smaller than a day.</td><td>*Four hours*</td></tr>
<tr><td>`PERCENT`</td><td>Percentage, including "%".</td><td>*Eighty percent*</td></tr>
<tr><td>`MONEY`</td><td>Monetary values, including unit.</td><td>*Twenty Cents*</td></tr>
<tr><td>`QUANTITY`</td><td>Measurements, as of weight or distance.</td><td>*Several kilometers, 55kg*</td></tr>
<tr><td>`ORDINAL`</td><td>"first", "second", etc.</td><td>*9th, Ninth*</td></tr>
<tr><td>`CARDINAL`</td><td>Numerals that do not fall under another type.</td><td>*2, Two, Fifty-two*</td></tr>
</table>

For more on **Named Entity Recognition** visit https://spacy.io/usage/linguistic-features#101

For **noun_chunks** visit https://spacy.io/usage/linguistic-features#noun-chunks


# Visualizing Named Entities
Besides viewing Part of Speech dependencies with `style='dep'`, **displaCy** offers a `style='ent'` visualizer:

In [ ]:
# Import the displaCy library
from spacy import displacy

In [ ]:
#displacy.render(doc.nlp, style='ent', jupyter=True)


## Viewing Sentences Line by Line
Unlike the **displaCy** dependency parse, the NER viewer has to take in a Doc object with an `ents` attribute. For this reason, we can't just pass a list of spans to `.render()`, we have to create a new Doc from each `span.text`:



In [ ]:
for sent in doc.nlp:
    displacy.render(nlp(doc.nlp), style='ent', jupyter=True)

TypeError: ignored

Customizing Colors and Effects
You can also pass background color and gradient options:

In [ ]:
colors = {'ORG': 'linear-gradient(90deg, #aa9cfc, #fc9ce7)', 'PRODUCT': 'radial-gradient(yellow, green)'}

options = {'ents': ['ORG', 'PRODUCT'], 'colors':colors}

displacy.render(doc, style='ent', jupyter=True, options=options)

ValueError: ignored

___
# Creating Visualizations Outside of Jupyter
If you're using another Python IDE or writing a script, you can choose to have spaCy serve up HTML separately.

Instead of `displacy.render()`, use `displacy.serve()`:

In [ ]:
displacy.serve(doc.nlp, style='ent', options=options)

## MY OWN MODEL (POI CANCELLARE)


spaCy’s models are statistical and every “decision” they make – for example, which part-of-speech tag to assign, or whether a word is a named entity – is a prediction. This prediction is based on the examples the model has seen during training. To train a model, you first need training data – examples of text, and the labels you want the model to predict. This could be a part-of-speech tag, a named entity or any other information.

All spaCy models support online learning, so you can update a pretrained model with new examples. You’ll usually need to provide many examples to meaningfully improve the system — a few hundred is a good start, although more is better.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
doc=pd.read_table('gdrive/My Drive/TESI/SPACY/train.txt')
test_data=pd.read_table('gdrive/My Drive/TESI/SPACY/test.txt')
valid_data=pd.read_table('gdrive/My Drive/TESI/SPACY/valid.txt')

https://towardsdatascience.com/custom-named-entity-recognition-using-spacy-7140ebbb3718


In [ ]:
import pandas as pd
import numpy as np
df= pd.read_csv('gdrive/My Drive/TESI/SPACY/train.txt', sep=" ", encoding="latin1") #ho messo insieme in un unico file train.txt+test.txt
df.columns = ['Word','POS','POS2','Tag']
df.drop('POS2',axis=1,inplace=True)
NaN=np.nan
df['Sentence #']= NaN
df[['Sentence #']]=df[["Sentence #"]].astype(object)
#Let us take a sneak-peak into the dataset first
df.head()


In [ ]:
c = list()
for i in range(len(df)):
  #print(df['Word'][i])
  if df['Word'][i]=='.':
    c.append('Sentence:')
  else:
    c.append(NaN)
    

In [ ]:
#Tolgo ultimo elemento e aggiungo un elemento in cima alla lista cosi scifta tutto di una posizione (questo perchè ho bisgono che Sentence: 1 sia alla prima parola della frase e non dove c'è il punto )
c.pop()
c.insert(0,'Sentence:')

In [ ]:
sen = 'Sentence:'
count = 1
for i in range(len(c)):
  if c[i]=='Sentence:':
    c[i] = sen+str(count)
    count+=1

In [ ]:
c


In [ ]:
df['Sentence #']=c

In [ ]:
df

In [ ]:
df.drop("POS",axis=1,inplace=True)

In [ ]:
df.drop("Sentence #",axis=1,inplace=True)

In [ ]:
df

In [ ]:
df.to_csv("gdrive/My Drive/TESI/SPACY/df.txt",header=True,index=False)

In [ ]:
label_list=["B-ORG","O","B-MISC","B-PER","B-LOC","I-ORG","I-MISC","I-PER","I-LOC"]

In [ ]:
# Convert .tsv file to dataturks json format. 
import json
import logging
import sys
def tsv_to_json_format(input_path,output_path,unknown_label):
    try:
        f=open(input_path,'rb') # input file
        fp=open(output_path, 'wb') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        for line in f:
            if line[0:len(line)-1]!='.\tO':
                word,entity=line.split('\t')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None

tsv_to_json_format("gdrive/My Drive/TESI/SPACY/dataframe.tsv","gdrive/My Drive/TESI/SPACY/converted.json",label_list)

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
#PROVA 
# Convert json file to spaCy format.
import plac
import logging
import argparse
import sys
import os
import json
import pickle

@plac.annotations(input_file=("Input file", "option", "i", str), output_file=("Output file", "option", "o", str))

def main(input_file=None, output_file=None):
    try:
        training_data = []
        lines=[]
        with open(input_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        print(training_data)

        with open(output_file, 'wb') as fp:
            pickle.dump(training_data, fp)

    except Exception as e:
        logging.exception("Unable to process " + input_file + "\n" + "error = " + str(e))
        return None
if __name__ == '__main__':
    plac.call(main)


In [ ]:
# Convert .tsv file to dataturks json format. 
import json
import logging
import sys
# def tsv_to_json_format(input_path,output_path,unknown_label):
    try:
        f=open(input_path,'r') # input file
        fp=open(output_path, 'w') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        for line in f:
            if line[0:len(line)-1]!='.\tO':
                word,entity=line.split('\t')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None

tsv_to_json_format("Data/ner_corpus_260.tsv",'Data/ner_corpus_260.json','abc')

In [ ]:
def load_data_spacy(file_path):
    ''' Converts data from:
    label \t word \n label \t word \n \n label \t word
    to: sentence, {entities : [(start, end, label), (stard, end, label)]}
    '''
    file = open(file_path, 'r')
    training_data, entities, sentence, unique_labels = [], [], [], []
    current_annotation = None
    end = 0 # initialize counter to keep track of start and end characters
    for line in file:
        line = line.strip(".").split(" ")
        # lines with len > 1 are words
        if len(line) > 1:
            label = line[0][2:]     # the .txt is formatted: label \t word, label[0:2] = label_type
            label_type = line[0][0] # beginning of annotations - "B", intermediate - "I"
            word = line[1]
            sentence.append(word)
            end += (len(word) + 1)  # length of the word + trailing space
           
            if label_type != 'I' and current_annotation:  # if at the end of an annotation
                entities.append((start, end - 2 - len(word), current_annotation))  # append the annotation
                current_annotation = None                 # reset the annotation
            if label_type == 'B':                         # if beginning new annotation
                start = end - len(word) - 1  # start annotation at beginning of word
                current_annotation = label   # append the word to the current annotation
            if label_type == 'I':            # if the annotation is multi-word
                current_annotation = label   # append the word
           
            if label != 'O' and label not in unique_labels:
                unique_labels.append(label)
 
        # lines with len == 1 are breaks between sentences
        if len(line) == 1:
            if current_annotation:
                entities.append((start, end - 1, current_annotation))
            sentence = " ".join(sentence)
            training_data.append([sentence, {'entities' : entities}])
            # reset the counters and temporary lists
            end = 0            
            entities, sentence = [], []
            current_annotation = None
    file.close()
    return training_data, unique_labels            
           
TRAIN_DATA, LABELS = load_data_spacy("gdrive/My Drive/TESI/SPACY/df2.txt")

In [ ]:
[x[0] for x in TRAIN_DATA[1:10]]

In [ ]:
doc1=nlp.pipe(df['Word'].astype('str').values, batch_size=50,
                        n_threads=3)
doc2=list(doc1)


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
for doc in nlp.pipe(df['-DOCSTART-'].astype('str').values, batch_size=50,n_threads=3):
    # Do something with the doc here
    print([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
doc2

In [ ]:
df.head()

In [ ]:
a=df
a

In [ ]:
data=df[['Word','Tag']]
data

In [ ]:
pip install literal

In [ ]:
import ast

In [ ]:
data.to_csv('gdrive/My Drive/TESI/SPACY/data.csv')

In [ ]:
def getcharoffsetsfromwordoffsets(doc,entities):
    charoffsets = []
    for entity in entities:
        span = doc[entity[0]:entity[1]]
        charoffsetentitytuple = (span.start_char, span.end_char, entity[2])
        charoffsets.append(charoffsetentitytuple)
    return charoffsets

def convertspacyapiformattocliformat(nlp, TRAIN_DATA):
    docnum = 1
    documents = []
    for t in TRAIN_DATA:
        doc = nlp(t[0])
        charoffsetstuple = getcharoffsetsfromwordoffsets(doc,t[1]['entities'])
        tags = biluo_tags_from_offsets(doc, charoffsetstuple)
        ner_info = list(zip(doc, tags))
        tokens = []
        sentences = []
        for n, i in enumerate(ner_info):
            token = {"head" : 0,
            "dep" : "",
            "tag" : "",
            "orth" : i[0].string,
            "ner" : i[1],
            "id" : n}
            tokens.append(token)
        sentences.append({'tokens' : tokens})
        document = {}
        document['id'] = docnum
        docnum+=1
        document['paragraphs'] = []
        paragraph = {'raw': doc.text,"sentences" : sentences}
        document['paragraphs'] = [paragraph]
        documents.append(document)
    return documents

In [ ]:
getcharoffsetsfromwordoffsets(doc2,entities)

In [ ]:
# Convert .tsv file to dataturks json format. 
import json
import logging
import sys
def tsv_to_json_format(input_path,output_path,unknown_label):
    try:
        f=open(input_path,'r') # input file
        fp=open(output_path, 'w') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        for line in f:
            if line[0:len(line)-1]!='.\tO':
                word,entity=line.split('\n')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None

tsv_to_json_format('gdrive/My Drive/TESI/SPACY/data.csv','gdrive/My Drive/TESI/SPACY/ner_corpus_260.json','abc')

In [ ]:
# Convert json file to spaCy format.
import plac
import logging
import argparse
import sys
import os
import json
import pickle

#@plac.annotations(input_file=("Input file", "option", "i", str), output_file=("Output file", "option", "o", str))

def main(input_file=None, output_file=None):
    try:
        training_data = []
        lines=[]
        with open(input_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        print(training_data)

        with open(output_file, 'wb') as fp:
            pickle.dump(training_data, fp)

    except Exception as e:
        logging.exception("Unable to process " + input_file + "\n" + "error = " + str(e))
        return None
if __name__ == '__main__':
    plac.call(main)